<a href="https://github.com/gutris1/segsmaker">
  <img alt="GitHub repo" src="https://img.shields.io/badge/GitHub-6e5494?style=for-the-badge&logo=github&logoColor=white"/>
</a>

**webui list** = A1111, Forge, ComfyUI, ReForge, SwarmUI<br>
**sd list** = 1.5 or xl<br>
**civitai_key** = get your civitai api key from [here](https://civitai.com/user/account)<br>
**hf_read_token** = for downloading models from your private repo (**optional**)

### Webui Installer

In [ ]:
# @markdown ### <br> **WebUI configuration**

webui = 'A1111' # @param ["A1111", "Forge", "ComfyUI", "ReForge", "SwarmUI"]
sd = 'xl' # @param ["1.5", "xl"]
civitai_key = '' # @param {'type': 'string'}
hf_read_token = '' # @param {'type': 'string'}

!curl -sLo /content/setup.py https://github.com/gutris1/segsmaker/raw/main/script/KC/setup.py
%run /content/setup.py --webui="$webui" --sd="$sd" --civitai_key="$civitai_key" --hf_read_token="$hf_read_token"

In [ ]:
''' SD Extensions / ComfyUI Custom Nodes '''
%cd -q $Extensions
!git clone

''' VAE '''
%cd -q $VAE
%download

''' Embeddings '''
%cd -q $Embeddings
%download

''' Upscalers '''
%cd -q $Upscalers
%download

In [ ]:
''' Checkpoint '''
%cd -q $CKPT
%download https://civitai.com/api/download/models/785352 bigblu25dmix25DStyle_v10.safetensors

''' LoRA '''
%cd -q $LORA
%download https://civitai.com/models/122359/detail-tweaker-xl
%download https://civitai.com/models/669571/pony-add-more-details

In [ ]:
''' Controlnet '''
%run $Controlnet_Widget

### Launcher
args list :
-  **A1111** = `--xformers`
- **Forge** = `--disable-xformers --opt-sdp-attention --cuda-stream --pin-shared-memory`
- **ComfyUI** = `--dont-print-server --preview-method auto --use-pytorch-cross-attention`
- **ReForge** = `--xformers --cuda-stream --pin-shared-memory`
- **SwarmUI** = `--launch_mode none`
<br><br>

For ComfyUI, add `--skip-comfyui-check` to skip checking custom node dependencies

In [ ]:

config_option = "A1111" # @param ["A1111", "Forge", "ComfyUI", "ReForge", "SwarmUI"]

if config_option == "ComfyUI":
    config_arg = "--dont-print-server --preview-method auto --use-pytorch-cross-attention"
else if config_option == "Forge":
    config_arg = "--disable-xformers --opt-sdp-attention --cuda-stream --pin-shared-memory"
else if config_option == "ReForge":
    config_arg = "--xformers --cuda-stream --pin-shared-memory"
else if config_option == "SwarmUI":
    config_arg = "--launch_mode none"

# Move to the WebUI directory
%cd -q $WebUI

# Run the segsmaker script with the selected configuration
command = f"%run segsmaker.py --{config_arg}"
print(f"Running: {command}")
%run segsmaker.py --{config_arg}
